# Getting started

If this is the first time that you're seeing this "notebook" then you will need to download this notebook to your computer and install Jupyter to run this notebook.

1. Download this Jupyter notebook and save it to your desktop.
   If you are viewing this notebook as a Gist on GitHub, use the "Download ZIP" button that you'll find to the right.
2. Open a command console and install Jupyter by running the following command:

        pip install jupyter urllib3
3. From the same command console as above, "cd" to your desktop and then start the Jupyter notebook server:

        cd Desktop
        jupyter notebook
4. This will now open a web browser.
5. Open the `Oktane15-platform-lab.ipynb` file
6. Select the first "cell" in this notebook ("Getting started"), then press the "Play" or "Run cell" button as you read each cell. This button doesn't do anything on cells with text, but it will execute the Python code in cells that contain Python.

# Using Python to run reports from Okta

Goal for this notebook is to teach you how to generate the following types of reports from Okta using the [Okta API](http://developer.okta.com) and the [Python](https://en.wikipedia.org/wiki/Python_%28programming_language%29) programming language. It's okay if you don't know Python, just follow these instructions and read through the code as you go along, press the "Play" button after you read a snippet of code to see what happens when the code is executed.

In this notebook, you will learn how to generate these reports:
 - List users who have been updated in the past 24 hours
 - List users with failed logins
 
We will cover the two reports above. Once you are done with those reports, you can challenge yourself to see if you can create thise report:
 - List users who have logged into an application
 
 
Let's get started!
 
First we will include the Python libraries that we will need to run these examples. We include `os` to read out the `OKTA_URL` and `OKTA_API_KEY` environment variables, `datetime` is used for calculating times used in fetching the past 24 hours of data, `okta` imports the Python SDK for the Okta API, and lastly, we include `urllib3` and run the `disable_warnings()` method to disable SSL warnings for the purposes of this demo - something you must *never* do in production.

In [1]:
import os
import datetime
from datetime import timedelta

import okta
import urllib3
urllib3.disable_warnings()

Next, we will get the values for `base_url` and `api_token` from your environment variables. 

If you don't have these defined already, just replace everything after the `=` sign with the approprate value for your Okta org. Here is an example of what this might look like for you:

    base_url = "https://example.oktapreview.com"
    api_token = "00Ab0cDEfghiJK1LMnOpQrSt2uvw3XyZABCdeFgHIJ"

In [2]:
base_url = os.environ['OKTA_URL']
api_token = os.environ['OKTA_API_KEY']

First we instantiate the `UsersClient` with the values of the `base_url` and `api_token` variables that we defined above:

In [3]:
usersClient = okta.UsersClient(base_url, api_token)

Next, we fetch a list of users from our Okta org using the `get_users()` method:

In [4]:
users = usersClient.get_users()

Now we've created a variable named `users`, which is an array of Okta user objects. 

Let's take a look at what the first element in that array and see what a user object in your Okta org looks like:

In [5]:
user = users[0]
user.__dict__

{'activated': None,
 'created': datetime.datetime(2015, 3, 6, 18, 58, 37, tzinfo=tzutc()),
 'credentials': <okta.models.user.LoginCredentials.LoginCredentials instance at 0x1101daf80>,
 'id': u'00u3i341lhaqfLxcD0h7',
 'lastLogin': None,
 'lastUpdated': datetime.datetime(2015, 10, 19, 23, 3, 51, tzinfo=tzutc()),
 'links': {u'activate': <okta.models.Link.Link instance at 0x1105b01b8>,
  u'deactivate': <okta.models.Link.Link instance at 0x1105b0170>},
 'passwordChanged': None,
 'profile': <okta.models.user.UserProfile.UserProfile instance at 0x1101daea8>,
 'status': u'STAGED',
 'statusChanged': None,
 'transitioningToStatus': None}

Notice in particular the `lastUpdated` and `profile` properties.

`lastUpdated` contains the date that this user was last updated. 
`profile` contains the user profile as a "dictionary" which includes the `firstName` and `lastName` properties.

Let's take a look at the `lastUpdated` time for the first 10 users in the `users` array by looping over the first ten users and the printing out the `lastUpdated` property for each user:

In [6]:
num_results = 10
for user in users[0:num_results]:
    print user.lastUpdated
    

2015-10-19 23:03:51+00:00
2015-03-06 19:02:41+00:00
2015-03-10 22:42:44+00:00
2015-06-16 18:27:08+00:00
2015-06-16 18:28:05+00:00
2015-08-19 00:08:27+00:00
2015-08-19 00:08:27+00:00
2015-08-19 00:08:59+00:00
2015-08-19 00:09:00+00:00
2015-08-19 00:09:00+00:00


## Listing users who have been updated in the past 24 hours

Now we're ready to try getting the list of "users who have been updated in the past 24 hours".

Here is what we're going to do:
1. Learn the [syntax for asking Okta to do filtering on our behalf](http://developer.okta.com/docs/api/resources/users.html#filters).
2. Translate that syntax to the `usersClient.get_users()` method.

Take a moment now to read the [syntax for filtering objects in the Okta API](http://developer.okta.com/docs/api/resources/users.html#filters).

Upon consulting the filtering syntax for the `/users` Okta resource, we find that there is an example of [listing users updated after a timestamp](http://developer.okta.com/docs/api/resources/users.html#list-users-updated-after-timestamp), using this example, we can see that we'll need to update the `usersClient.get_users()` method with that syntax, here is what it might look like: 

    usersClient.get_users(filter_string='lastUpdated gt "2013-07-01T00:00:00.000Z"')

What we see immediately is that the timestamp is in UTC ([because it ends in 'Z'](https://tools.ietf.org/html/rfc3339#section-2)). It also becomes apparent that we will need a way to generate the current timestamp.

Let's do that now. Below we use the `datetime` module to get the UTC time for "now", subtract 24 hours from "now" and then format the "24 hours before now" time into the RFC3339 format:

In [7]:
now = datetime.datetime.utcnow()
one_day = timedelta(hours=24)
past = now - one_day
filter_24_hrs_ago = past.strftime('lastUpdated gt "%Y-%m-%dT%H:%M:%S.000Z"')
print "Filter: '{}'".format(filter_24_hrs_ago)

Filter: 'lastUpdated gt "2015-10-26T22:22:23.000Z"'


Now that we have the `filter_24_hrs_ago` filter string, let's use it to ask the Okta API for a new list of users:

In [8]:
recent_users = usersClient.get_users(filter_string=filter_24_hrs_ago)

Finally, let's take a look at the users that we've modified in the past 24 hours, printing out the `id`, `profile.firstName`, and `profile.lastName` for each user:

In [9]:
print "Number of users found: {}".format(len(recent_users))
for user in recent_users:
    print "{},{},{}".format(user.id, user.profile.firstName, user.profile.lastName)

Number of users found: 1
00u2ukqmqpOMWXKCPODF,Bugs,Bunny


**Note:** If you see "`Number of users found: 0`" above, then you should modify one of the users in your Okta org and then re-execute the cell above.

There we go! You now have a list of users that have been modified in the last 24-hours. Now let's take what we've learned, and apply it to events.

## Listing users with failed logins

We will start by creating an API client for the [Okta Events API](http://developer.okta.com/docs/api/resources/events.html), we will be re-using the `base_url` and `api_token` variables that we defined earlier:

In [10]:
eventsClient = okta.EventsClient(base_url, api_token)

Next, let's fetch the most recent event and display it, so we can see what it looks like:

In [11]:
last_event = eventsClient.get_events(limit=1)[0]
last_event.__dict__

{'action': <okta.models.event.Action.Action instance at 0x1101f5dd0>,
 'actors': [<okta.models.event.Actor.Actor instance at 0x1101f5b48>,
  <okta.models.event.Actor.Actor instance at 0x1101f5c68>],
 'eventId': u'tevuevWybuvRmmX4ltIBVxthw1405960621000',
 'id': None,
 'published': datetime.datetime(2014, 7, 21, 16, 37, 1, tzinfo=tzutc()),
 'requestId': u'reqm0mx-EP9RqmgUXyP59aPZg',
 'sessionId': u'trs-FXbgw0ESs2iCRCdbu3isg',
 'targets': None}

Since this doesn't show the full object, let's take a look at an [example event from the Okta developer documentation](http://developer.okta.com/docs/api/resources/events.html#example):

    {
       "eventId":"tevGr2BhQTMR72OiBGvKXTp2Q1399593071000",
       "published":"2014-05-08T23:51:11.000Z",
       "requestId":"req8U_MHmEbSai_0I4RopTnfA",
       "sessionId":"000cWiYg47QSFyk1YjE6cDcEg",
       "action":{
          "message":"Okta user created",
          "categories":[
             "User Creation"
          ],
          "objectType":"core.user.config.user_creation.success",
          "requestUri":"Background"
       },
       "actors":[
          {
             "id":"00ue1aWYUCUFFKXLXELW",
             "displayName":"Add-Min O'Cloudy Tud",
             "login":"administrator1@clouditude.net",
             "objectType":"User"
          },
          {
             "id":"Jakarta Commons-HttpClient/3.1",
             "displayName":"UNKNOWN",
             "ipAddress":"",
             "objectType":"Client"
          }
       ],
       "targets":[
          {
             "id":"00ue1gAKBMCSWHRZYDJS",
             "displayName":"Inca-Louise O'Rain Dum",
             "login":"inca@clouditude.net",
             "objectType":"User"
          }
       ]
    }

As you can see, event objects contain a lot of information. However, all that we need for this example is the `published` property (the timestamp when event was published), the `action.objectType` property (the unique action type), and the `targets[0].displayName` property (the name of the event target).

Let's take a look at the contents of the `published` and `action.objectType` properties:

In [12]:
last_event.published

datetime.datetime(2014, 7, 21, 16, 37, 1, tzinfo=tzutc())

In [13]:
last_event.action.objectType

u'app.app_instance.change'

Now let's get a failed login from the logs. 

By now, you should be familiar with the filtering syntax for Okta, since you used that syntax when you found a list of updated users above.

If you look at the [filter examples in the Events API documentation](http://developer.okta.com/docs/api/resources/events.html#filters) you will see that you can use `action.objectType eq ":actionType"` to filter events with a specific objectType. If you look at the [documentation for the User Authentication object type](http://developer.okta.com/docs/api/resources/events.html#user-authentication), you will find that the objectType you want is `core.user_auth.login_failed`.

Combining these together, we get this filter: `action.objectType eq "core.user_auth.login_failed"` - so let's use this filter and get a list of the last 10 failed login attempts:

In [14]:
failed_logins = eventsClient.get_events(limit=10, filter_string='action.objectType eq "core.user_auth.login_failed"')
failed_logins

[<okta.models.event.Event.Event instance at 0x11054b368>,
 <okta.models.event.Event.Event instance at 0x11054b440>,
 <okta.models.event.Event.Event instance at 0x11054b680>,
 <okta.models.event.Event.Event instance at 0x11054b8c0>,
 <okta.models.event.Event.Event instance at 0x11054bb00>,
 <okta.models.event.Event.Event instance at 0x11054bd40>,
 <okta.models.event.Event.Event instance at 0x11055b050>,
 <okta.models.event.Event.Event instance at 0x11055b200>,
 <okta.models.event.Event.Event instance at 0x11055b440>,
 <okta.models.event.Event.Event instance at 0x11055b680>]

Note: If you see an empty list above "`[]`", you will need to make some failed login attempts against your Okta org.

Now that we have a list of failed login attempts, let's print out the event ID, the time of the event, and the display name for the user with the failed login event:

In [15]:
for failed_login in failed_logins:
    print u"{} {} {}".format(failed_login.eventId,
                            failed_login.published,
                            failed_login.targets[0].displayName)

tev40KZEammQt-PyC4ALGUK0A1406049331000 2014-07-22 17:15:31+00:00 Joël Franusic
tevMqaPW_HBR42ET2a-rwb7wQ1406064257000 2014-07-22 21:24:17+00:00 None
tevL1bNluevT4i_UwwhdtbURA1406064467000 2014-07-22 21:27:47+00:00 None
tevzzxBBQoqTYKevTdoCPIjKg1406064581000 2014-07-22 21:29:41+00:00 None
tevCIsKlbm6QaiuSqLYWgoqnQ1406064585000 2014-07-22 21:29:45+00:00 None
tevpZuhT4znS-q0qcuCK3sc5A1406064610000 2014-07-22 21:30:10+00:00 None
tev5K23nxi5R0OhDFNqNKet_g1406064792000 2014-07-22 21:33:12+00:00 None
tevd0U-0qXGTSG9dFbULUeTGA1406064794000 2014-07-22 21:33:14+00:00 None
tevEfSB7j9UQfSC831nLbx_vg1406064835000 2014-07-22 21:33:55+00:00 None
tev-xBSKBmoS4eXTD9ZyAJLBQ1406593163000 2014-07-29 00:19:23+00:00 Joël Franusic


And there you have it: the last ten (or less) failed login events against your Okta org.

## Listing users who have logged into a specific application

This is an "extra credit" section. See if you can figure out how to get a list of users that have logged into a specific application (like Salesforce). 

*Hint:* In your research, you should "target" the [app.auth.sso](http://developer.okta.com/docs/api/resources/events.html#application-authentication) object type.